In [2]:
#!pip install kaleido
!pip install transformers[torch]
#!pip install typing-extensions==3.10.0.2
#!pip install gradio

  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/c1/bd/f64d67df4d3b05a460f281defe830ffab6d7940b7ca98ec085e94e024781/transformers-4.34.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/121.5 kB ? eta -:--:--
     --- ------------------------------------ 10.2/121.5 kB ? eta -:--:--
     --------- --------------------------- 30.7/121.5 kB 660.6 kB/s eta 0:00:01
     ------------ ------------------------ 41.0/121.5 kB 281.8 kB/s eta 0:00:01
     ---------------------------- -------- 92.2/121.5 kB 525.1 kB/s eta 0:00:01
     ------------------------------------ 121.5/121.5 kB 595.2 kB/s eta 0:00:00
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/b8/ad/3398312096118c4e62a5827664e52a04d5068e84d04142dd4a0da8a567ae/regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     -------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ------------ --------------------------- 1.9/5.7 MB 1.5 MB/s eta 0:00:03
     ------------- -------------------------- 2.0/5.7 MB 1.5 MB/s eta 0:00:03
     -------------- ------------------------- 2.1/5.7 MB 1.6 MB/s eta 0:00:03
     --------------- ------------------------ 2.2/5.7 MB 1.6 MB/s eta 0:00:03
     --------------- ------------------------ 2.3/5.7 MB 1.6 MB/s eta 0:00:03
     ---------------- ----------------------- 2.3/5.7 MB 1.6 MB/s eta 0:00:03
     ----------------- ---------------------- 2.5/5.7 MB 1.6 MB/s eta 0:00:02
     ------------------ --------------------- 2.6/5.7 MB 1.7 MB/s eta 0:00:02
     ------------------- -------------------- 2.8/5.7 MB 1.7 MB/s eta 0:00:02
     -------------------- ------------------- 2.9/5.7 MB 1.8 MB/s eta 0:00:02
     --------------------- ------------------ 3.1/5.7 MB 1.8 MB/s eta 0:00:02
     ---------------------- ----------------- 3.2/5.7 MB 1.8 MB/s eta 0:00:02
     ----------------------- ---------------- 3.3/5.7 MB 1.9 MB/

In [6]:
#!pip install gradio --upgrade


In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import gradio as gr

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)


model_path = f"Azie88/sentiment_analysis_model_trainer"
#tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [7]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


def sentiment_analysis(text):
    text = preprocess(text)

    # PyTorch-based models
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores_ = output[0][0].detach().numpy()
    scores_ = softmax(scores_)

    # Format output dict of scores
    labels = ['Negative', 'Neutral', 'Positive']
    scores = {l:float(s) for (l,s) in zip(labels, scores_) }

    return scores

In [8]:
demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(placeholder="Write your tweet here..."),
    outputs="label",
    # interpretation="default",
    examples=[["This is wonderful!"]])

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


c:\Users\Aobando\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\components\button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(
